In [4]:
from numpy import asarray, load, nanargmin, absolute
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from svm import SVM

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
files = os.listdir('.')
print(files)

['.config', 'drive', 'ubipr_vgg_flattened_embeddings.npz', '.ipynb_checkpoints', 'kernels.py', 'svm.py', 'sample_data']


In [5]:
# load dataset
data = load('ubipr_vgg_flattened_embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
i = -1
testX = list(testX)
testy = list(testy)
del testX[2911]
del testy[2911]
testX = asarray(testX)
testy = asarray(testy)
print(trainy.shape)
print(testy.shape)

(6967,)
(3231,)


In [6]:
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
print(testy)
trainy = out_encoder.transform(trainy)
print(trainy)
testy = out_encoder.transform(testy)
print(testy)


['1_L' '1_L' '1_L' ... '99_R' '99_R' '99_R']
[110 110 110 ... 503 503 503]
[110 110 110 ... 503 503 503]


In [7]:
import cupy as xp 
# Move data to GPU
x_train = xp.asarray(trainX)
x_test =  xp.asarray(testX)
y_train = xp.asarray(trainy)
y_test = xp.asarray(testy)

In [8]:
# fit model
# from sklearn
# model = SVC(kernel='linear', probability=True, verbose = True)
model = SVM(kernel='linear', kernel_params={})

In [9]:
# print(len(newX))
# newX
model.fit(x_train, y_train)

In [11]:
import pickle
print("Saving")
filename = '/content/drive/My Drive/And/ubipr_vgg16_flattened2_svm_trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Saving


In [12]:
yhat_train = model.predict(x_train)
yhat_test = model.predict(x_test)
# score

# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

ValueError: ignored

In [22]:
from cupy import asnumpy
ny_train = asnumpy(y_train)
ny_test = asnumpy(y_test)
nyhat_train = asnumpy(yhat_train)
nyhat_test = asnumpy(yhat_test)

In [23]:
score_train = accuracy_score(ny_train, nyhat_train)
score_test = accuracy_score(ny_test, nyhat_test)

In [25]:
score_test

0.9387186629526463

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(testy, yhat_test, pos_label=1)
fnr = 1 - tpr
print(fpr[nanargmin(absolute((fnr - fpr)))])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 25)
rf.fit(trainX, trainy)

In [ ]:
import pickle
print("Saving")
filename = 'ubipr_vgg16_rf_trained_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
yhat_train = rf.predict(trainX)
yhat_test = rf.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(testy, yhat_test, pos_label=1)
fnr = 1 - tpr
print(1-fpr[nanargmin(absolute((fnr - fpr)))])

In [ ]:
import numpy as np
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=25, metric = cosine_similarity)
knn.fit(trainX, trainy)

In [ ]:
yhat_train = knn.predict(trainX)
yhat_test = knn.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
trainX[0][0]